In [1]:
import json
import re
from collections import Counter
import pandas as pd
def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data    

In [12]:
# load jsonl file to list of dictionaries
# data_path='../data/shared/6000_posts_sample.jsonl'
data_path = '6000_posts_sample.jsonl'
data=load_jsonl(data_path)

Loaded 6000 records from 6000_posts_sample.jsonl


In [13]:
# put all tags in a single list
tags_all = []
for i in data:   
    tags = i['tags']
    tags_all.extend(tags)
print('Total tags number is:',len(tags_all))

Total tags number is: 20947


In [14]:
#find posts contain tags related to aws or amazon
tags_amazon = []
for i in data:   
    tags = i['tags']
    if 'amazon' or 'aws' in tags:
        res = [x for x in tags if re.search('amazon',x)]
        tags_amazon.extend(res)
        res2 = [x for x in tags if re.search('aws',x)]
        tags_amazon.extend(res2) 
counts_before_clean = len(tags_amazon)
print('Tags counts before cleaning:',counts_before_clean)

Tags counts before cleaning: 11446


In [15]:
# drop the useless tags
for item in tags_amazon:
    if  item == 'amazon' or item =='amazon-web-services':
        tags_amazon.remove(item)
counts_after_clean = len(tags_amazon)
print('Tags counts after withdraw <amazon> and <amazon-web-services>:',counts_after_clean)

Tags counts after withdraw <amazon> and <amazon-web-services>: 7743


In [16]:
key= Counter(tags_amazon).keys()
value=Counter(tags_amazon).values()
tags_dict = dict(zip(key,value))

In [17]:
# try to find features with high frequency
def frequent_features(input_num):
    num = input_num
    frequent_feature = {}
    for item in tags_dict.keys():
        if tags_dict[item]>num:
            frequent_feature[item]=tags_dict[item]
    return frequent_feature

In [18]:
frequent_features(100)

{'amazon-cognito': 175,
 'amazon-elastic-beanstalk': 306,
 'amazon-cloudformation': 194,
 'aws-lambda': 687,
 'amazon-s3': 1057,
 'amazon-redshift': 160,
 'amazon-iam': 101,
 'aws-cli': 115,
 'amazon-dynamodb': 382,
 'amazon-ecs': 111,
 'amazon-ec2': 972,
 'amazon-rds': 131,
 'aws-api-gateway': 181,
 'amazon-sqs': 106,
 'aws-sdk': 203,
 'amazon-cloudfront': 155,
 'amazon-web-services': 590}

In [19]:
# sort tags by frequency
tags_dict_sorted = {k:v for k,v in sorted(tags_dict.items(),key=lambda item:item[1],reverse = True)}

In [20]:
# output the result
sorted_service = pd.DataFrame(list(tags_dict_sorted.items()), columns = ['service','frequency'])

In [ ]:
sorted_service.to_csv("service_counts_6000_samples.csv")